## Pulling API Soccer Data for Analysis

### Requirements and Tools

#### Import Dependencies

In [1]:
import requests
import pandas as pd
import json
import credentials ## gitignore file

#### Parameters for Pull

In [2]:
headers = {
    'x-rapidapi-key': credentials.X_RapidAPI_Key,
    'x-rapidapi-host': credentials.X_RapidAPI_Host
    }

#### Pretty Printing Json

In [3]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

### League Pull

#### Pulling League Data

In [4]:
league_url = "https://api-football-v1.p.rapidapi.com/v2/leagues"
lr = requests.get(league_url, headers=headers)
print(lr.status_code)

200


#### Viewing League Json Data

In [5]:
league_json = lr.json()["api"]["leagues"]
league_one = league_json[0]
jprint(league_one)

{
    "country": "World",
    "country_code": null,
    "coverage": {
        "fixtures": {
            "events": true,
            "lineups": true,
            "players_statistics": true,
            "statistics": true
        },
        "odds": false,
        "players": true,
        "predictions": true,
        "standings": true,
        "topScorers": true
    },
    "flag": null,
    "is_current": 1,
    "league_id": 1,
    "logo": "https://media.api-sports.io/football/leagues/1.png",
    "name": "World Cup",
    "season": 2018,
    "season_end": "2018-07-15",
    "season_start": "2018-06-14",
    "standings": 1,
    "type": "Cup"
}


#### Putting League Json Data into Lists

In [6]:
league_lst = []
for l in league_json:
    league_lst.append([l['league_id'], l['name'], l['country']])

print(league_lst[:5])

[[1, 'World Cup', 'World'], [2, 'Premier League', 'England'], [3, 'Championship', 'England'], [4, 'Ligue 1', 'France'], [5, 'Ligue 2', 'France']]


#### Creating League DataFrame with List Data

In [7]:
league_df = pd.DataFrame(league_lst, columns=['league_id', 'league_name', 'league_country']).set_index('league_id')
league_df.head()
league_df.to_excel("Leagues.xlsx")

### Team Pull #1

#### Get last league pulled from the accumulated saved file

In [158]:
cumulated = pd.read_excel('team_df_save.xlsx')

s_num = cumulated['league_id'].iloc[-1] + 1
print (s_num)
display(cumulated)

188


,league_id,team_id,team_name,team_county,team_venue_name,team_city
0,1,1,Belgium,Belgium,Stade Roi Baudouin,Brussel
1,1,2,France,France,Stade de France,Paris
2,1,3,Croatia,Croatia,Stadion Maksimir,Zagreb
3,1,4,Russia,Russia,RZD Arena,Moskva
4,1,5,Sweden,Sweden,Friends Arena,Solna
...,...,...,...,...,...,...
4450,187,1466,Barranquilla,Colombia,Estadio Metropolitano Roberto Meléndez,Barranquilla
4451,187,1467,Valledupar,Colombia,Estadio Armando Maestre Pavajeau,Valledupar
4452,187,1468,Depor FC,Colombia,Estadio Olímpico Pascual Guerrero,Santiago de Cali
4453,187,1469,Orsomarso,Colombia,Estadio Francisco Rivera Escobar,Palmira


#### Loop through leagues API (for team data) the greatest amount of times allowed without having to pay :)

In [159]:
t_base_url = "https://api-football-v1.p.rapidapi.com/v2/teams/league/"
t_list = []
num = s_num

while num <= s_num + 29:
    t_url = t_base_url + str(num)
    tr = requests.get(t_url, headers=headers)
    t_list.append(tr)
    print (num) ## just for monitoring
    num += 1

188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217


In [160]:
## Just to preview the json
jprint(t_list[0].json())

{
    "api": {
        "results": 16,
        "teams": [
            {
                "code": null,
                "country": "Colombia",
                "founded": 1997,
                "is_national": false,
                "logo": "https://media.api-sports.io/football/teams/1145.png",
                "name": "Tigres FC",
                "team_id": 1145,
                "venue_address": "Tv 71D, Kennedy",
                "venue_capacity": 7800,
                "venue_city": "Bogot\u00e1, D.C.",
                "venue_name": "Estadio Metropolitano de Techo",
                "venue_surface": "grass"
            },
            {
                "code": null,
                "country": "Colombia",
                "founded": 1967,
                "is_national": false,
                "logo": "https://media.api-sports.io/football/teams/1146.png",
                "name": "Cortulua",
                "team_id": 1146,
                "venue_address": "Barrio Doce de Octubre",
                

#### Convert to json subscriptible and loop through data to pull out team info & add league info

In [161]:
team_list = []
league = s_num

for i in t_list:
    try:
        for x in i.json()["api"]["teams"]:
            team_list.append([league, x['team_id'], x['name'], x['country'], x['venue_name'], x['venue_city']])
    except:
        team_list.append('fucked api block') ## occasionally data doesn't come through
    league += 1

print(team_list)

[[188, 1145, 'Tigres FC', 'Colombia', 'Estadio Metropolitano de Techo', 'Bogotá, D.C.'], [188, 1146, 'Cortulua', 'Colombia', 'Estadio Doce de Octubre', 'Tuluá'], [188, 1147, 'Fortaleza FC', 'Colombia', 'Estadio Municipal de Cota', 'Cota'], [188, 1458, 'Bogota FC', 'Colombia', 'Estadio Metropolitano de Techo', 'Bogotá, D.C.'], [188, 1459, 'Real Cartagena', 'Colombia', 'Estadio Olímpico Jaime Morón León', 'Cartagena de Indias'], [188, 1460, 'Popayan', 'Colombia', 'Estadio Olímpico Pascual Guerrero', 'Santiago de Cali'], [188, 1461, 'Quindio', 'Colombia', 'Estadio Centenario', 'Armenia'], [188, 1462, 'Deportivo Pereira', 'Colombia', 'Estadio Hernán Ramírez Villegas', 'Pereira'], [188, 1463, 'Real Santander', 'Colombia', "Estadio de Fútbol Erwin O'Neil", 'San Andrés y Providencia'], [188, 1464, 'Llaneros', 'Colombia', 'Estadio Manuel Calle Lombana', 'Villavicencio'], [188, 1465, 'Union Magdalena', 'Colombia', 'Estadio Sierra Nevada', 'Santa Marta'], [188, 1466, 'Barranquilla', 'Colombia', 

#### Put listed data into a dataframe

In [162]:
team_df = pd.DataFrame(team_list, columns=['league_id', 'team_id', 'team_name', 'team_county', 'team_venue_name', 'team_city'])
display (team_df)

,league_id,team_id,team_name,team_county,team_venue_name,team_city
0,188,1145,Tigres FC,Colombia,Estadio Metropolitano de Techo,"Bogotá, D.C."
1,188,1146,Cortulua,Colombia,Estadio Doce de Octubre,Tuluá
2,188,1147,Fortaleza FC,Colombia,Estadio Municipal de Cota,Cota
3,188,1458,Bogota FC,Colombia,Estadio Metropolitano de Techo,"Bogotá, D.C."
4,188,1459,Real Cartagena,Colombia,Estadio Olímpico Jaime Morón León,Cartagena de Indias
...,...,...,...,...,...,...
837,217,1808,Caucaia W,Brazil,None,None
838,217,1809,Pinheirense W,Brazil,None,None
839,217,1810,Viana W,Brazil,None,None
840,217,1811,Santos W,Brazil,None,None


### Add Pulled Team Data to Historically Pulled Data

#### Add currently pulled data (team_df) to cummulated file

In [163]:
added = pd.concat([cumulated, team_df])
display(added)
added.to_excel('team_df_save.xlsx', index=False)

,league_id,team_id,team_name,team_county,team_venue_name,team_city
0,1,1,Belgium,Belgium,Stade Roi Baudouin,Brussel
1,1,2,France,France,Stade de France,Paris
2,1,3,Croatia,Croatia,Stadion Maksimir,Zagreb
3,1,4,Russia,Russia,RZD Arena,Moskva
4,1,5,Sweden,Sweden,Friends Arena,Solna
...,...,...,...,...,...,...
837,217,1808,Caucaia W,Brazil,None,None
838,217,1809,Pinheirense W,Brazil,None,None
839,217,1810,Viana W,Brazil,None,None
840,217,1811,Santos W,Brazil,None,None


### Merging League and Team Data

In [164]:
team_and_league_df = pd.merge(added, league_df, how='left', on='league_id')
display(team_and_league_df)

,league_id,team_id,team_name,team_county,team_venue_name,team_city,league_name,league_country
0,1,1,Belgium,Belgium,Stade Roi Baudouin,Brussel,World Cup,World
1,1,2,France,France,Stade de France,Paris,World Cup,World
2,1,3,Croatia,Croatia,Stadion Maksimir,Zagreb,World Cup,World
3,1,4,Russia,Russia,RZD Arena,Moskva,World Cup,World
4,1,5,Sweden,Sweden,Friends Arena,Solna,World Cup,World
...,...,...,...,...,...,...,...,...
5292,217,1808,Caucaia W,Brazil,None,None,Brasileiro Women,Brazil
5293,217,1809,Pinheirense W,Brazil,None,None,Brasileiro Women,Brazil
5294,217,1810,Viana W,Brazil,None,None,Brasileiro Women,Brazil
5295,217,1811,Santos W,Brazil,None,None,Brasileiro Women,Brazil
